## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

H:\1st-DL-CVMarathon\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [6]:

classifier=Sequential()

#卷積組合
input_shape = (32, 32, 3)
classifier.add(Convolution2D(filters=32, kernel_size=3, padding='same', input_shape=input_shape, activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())
'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(filters=64, kernel_size=3, padding='same', activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(128, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)















Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


H:\1st-DL-CVMarathon\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`




Epoch 1/100
50000/50000 [==============================] - 161s 3ms/step - loss: 1.2798 - acc: 0.5521
Epoch 2/100
50000/50000 [==============================] - 154s 3ms/step - loss: 0.8780 - acc: 0.6906
Epoch 3/100
50000/50000 [==============================] - 154s 3ms/step - loss: 0.7092 - acc: 0.7510
Epoch 4/100
50000/50000 [==============================] - 154s 3ms/step - loss: 0.5800 - acc: 0.7965
Epoch 5/100
50000/50000 [==============================] - 153s 3ms/step - loss: 0.4611 - acc: 0.8383
Epoch 6/100
50000/50000 [==============================] - 155s 3ms/step - loss: 0.3609 - acc: 0.8725
Epoch 7/100
50000/50000 [==============================] - 155s 3ms/step - loss: 0.2665 - acc: 0.9071
Epoch 8/100
50000/50000 [==============================] - 156s 3ms/step - loss: 0.1944 - acc: 0.9322
Epoch 9/100
50000/50000 [==============================] - 154s 3ms/step - loss: 0.1522 - acc: 0.9474
Epoch 10/100
50000/50000 [==============================] - 153s 3ms/step - loss

50000/50000 [==============================] - 151s 3ms/step - loss: 0.0194 - acc: 0.9939
Epoch 78/100
50000/50000 [==============================] - 150s 3ms/step - loss: 0.0178 - acc: 0.9943
Epoch 79/100
50000/50000 [==============================] - 152s 3ms/step - loss: 0.0140 - acc: 0.9959
Epoch 80/100
50000/50000 [==============================] - 152s 3ms/step - loss: 0.0197 - acc: 0.9940
Epoch 81/100
50000/50000 [==============================] - 152s 3ms/step - loss: 0.0130 - acc: 0.9958
Epoch 82/100
50000/50000 [==============================] - 151s 3ms/step - loss: 0.0177 - acc: 0.9951
Epoch 83/100
50000/50000 [==============================] - 151s 3ms/step - loss: 0.0177 - acc: 0.9946
Epoch 84/100
50000/50000 [==============================] - 152s 3ms/step - loss: 0.0196 - acc: 0.9936
Epoch 85/100
50000/50000 [==============================] - 154s 3ms/step - loss: 0.0164 - acc: 0.9952
Epoch 86/100
50000/50000 [==============================] - 159s 3ms/step - loss: 0.01

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [7]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[2.6095784e-04, 2.5079734e-12, 2.7820000e-09, 9.9971420e-01,
        2.2301012e-05, 3.7548015e-16, 2.0421080e-08, 2.0643467e-13,
        2.4875073e-06, 2.4892490e-12]], dtype=float32)